In [8]:
#Input: K: Number of dimensions
#       N: Approximation Parameter ('N' in the paper)
#       R: Vector-valued loss matrix input as a list (rows) of list (columns) of lists (vectors)
#       beta: Discount factor
#       rounds: Number of iterations ('n' in the paper)

#Output: V: List of H(K,N) points on the approximate Pareto frontier obtained after scaling losses in R so that the maximum absolute value of per-stage loss is (1-beta) on each dimension 
#        pol: List of H(K,N) distributions over actions (immediate randomized policy 'alpha' associated with each of the H(K,N) modes) 
#        trans: A list of H(K,N) lists. Each list is associated with a mode.
#               There are m entries in each list corresponding to m adversary actions. 
#               Each entry is a list of at most K tuples (a,b) where a is a mode and b is a probability of transitioning into that mode.                
#        modes: This is the list of H(K,N) modes. (This is included just for reference and is not needed in the implementation of the strategy)

from scipy.optimize import linprog
import copy

def ParetoFrontier(K,N,R,beta,rounds):
    #First, we scale the losses
    ma = max([max([abs(max(ont,key=abs)) for ont in ent]) for ent in R])
    factor = (1-beta)/ma
    R = [[[ont*factor for ont in ent] for ent in ola] for ola in R]
    m = len(R[1])
    el = len(R)
    
    #Next, we create the H(K,N) modes
    zero = [0 for i in range(K)]
    modes = []
    grid = [i/N for i in range(N+1)]
    basegrid = list(itertools.product(grid,repeat = K-1))
    basegrid = [list(ent) for ent in basegrid]  
    for i in range(K):
        temp = copy.deepcopy(basegrid)
        for ent in temp:
            ent.insert(i,0.0)
        modes += temp
    modes = [ent for ent in modes if ent != zero]
    modes = [zero] + modes 
    
    #Finally, we do value iteration
    V = [zero for ent in modes]
    for n in range(rounds):
        Vtemp = [[es*beta for es in ent] for ent in V]
        V = []
        pol = []
        trans = []
        for r in range(len(modes)):
            c = [1]+[0 for y in range(el+(len(modes)*m))]
            one = [[-1 for ent in range(K*m)]]
            Alph = []
            for i in range(el):
                temp = []
                for j in range(m):
                    temp += R[i][j]
                Alph.append(temp)
            Big = []
            for t in range(m):
                pre = [0 for h in range(t*K)]
                post = [0 for h in range((m-1-t)*K)]
                for ent in Vtemp:
                    Big.append(pre+ent+post)
            Aub = one + Alph + Big
            Aub = [list(ent) for ent in numpy.transpose(Aub)]
            bub = modes[r]*m
            Aeq = [[0]+[1 for y in range(el)]+[0 for y in range(len(modes)*m)]]
            for jj in range(m):
                Aeq.append([0 for y in range(el+1)]+[0 for y in range(jj*len(modes))]+[1 for y in range(len(modes))]+[0 for y in range((m-1-jj)*len(modes))])
            beq = [1] + [1 for y in range(m)]
            bnds = [(None,None)]+[(0,None) for i in range(len(c)-1)]
            sol = linprog(c, A_ub = Aub, b_ub = bub, A_eq = Aeq, b_eq = beq, bounds=bnds, options=dict(tol=10e-8))
            sol = sol.x.tolist()
            t = sol[0]
            V.append([ent+t for ent in modes[r]])
            tup = [[] for kk in range(m)]
            for e in range(el+1,len(sol)):
                if sol[e]>0:
                    tup[(e-el-1)//len(modes)].append(((e-el-1)%len(modes),sol[e]))
            trans.append(tup)
            pol.append(sol[1:el+1])
            
    return [V,pol,trans,modes]

In [ ]:
#sample application
K = 2
beta = [0.2,0.4]
N = [math.ceil(1/(0.05*(1-ent))) for ent in beta]
rounds = [math.ceil(math.log(0.01)/math.log(ent)) for ent in beta]
R = [[[0,1],[0,-1]],[[-1,0],[1,0]]]
Solutions = []
for i in range(len(beta)):
    Solutions.append(ParetoFrontier(K,N[i],R,beta[i],rounds[i]))